In [ ]:
import json

import pandas as pd

# ../src/manage.py dumpdata > dump.json

In [ ]:
with open("dump.json", "r") as f:
    dump = json.load(f)

ticket_objects = [record for record in dump if record["model"] == "tickets.jiraticket"]
for ticket in ticket_objects:
    ticket["fields"]["jira_ticket"] = ticket["pk"]
tickets = pd.DataFrame(ticket["fields"] for ticket in ticket_objects)[["jira_ticket", "assignee", "reporter"]]
tickets.head()

In [ ]:
annotation_objects = [record for record in dump if record["model"] == "tickets.annotation"]
annotations = pd.DataFrame(annotation["fields"] for annotation in annotation_objects)
annotations.head()

In [ ]:
annotated = tickets.merge(annotations, on="jira_ticket", how="inner").query("deliverable != '' and not no_deliverable")
annotated["Edit"] = '=HYPERLINK("https://deliverables.wiki/tickets/' + annotated.jira_ticket + '", "Edit")'
annotated["deliverable"] = '=HYPERLINK("' + annotated.deliverable + '", "Link")'

In [ ]:
annotated.head()

In [ ]:
PRODUCT_CHOICES = dict([
    ("desktop", "Firefox desktop"),
    ("fenix", "Firefox for Android (Fenix)"),
    ("fennec", "Firefox for Android (Fennec)"),
    ("fennec_ios", "Firefox for iOS"),
    ("other", "Other"),
    ("web", "Web properties"),
    ("many", "Many products"),
])

TOPICS = dict((
    ("accounts", "Accounts"),
    ("activity_stream", "Activity Stream"),
    ("addons", "Add-ons"),
    ("bookmarks", "Bookmarks"),
    ("devtools", "Devtools"),
    ("download", "Download page"),
    ("enterprise", "Enterprise"),
    ("geo", "Geolocation"),
    ("heartbeat", "Heartbeat"),
    ("nav", "Navigation"),
    ("newtab", "New tab"),
    ("marketing", "Marketing campaign"),
    ("onboarding", "Onboarding"),
    ("pwmgr", "Password manager"),
    ("payments", "Payments"),
    ("percperf", "Perceived performance"),
    ("perf", "Performance"),
    ("platform", "Platform"),
    ("pocket", "Pocket"),
    ("privacy", "Privacy"),
    ("search", "Search"),
    ("security", "Security"),
    ("services", "Services"),
    ("tp", "Tracking Protection"),
    ("vpn", "VPN"),
    ("", "")
))

ARTIFACTS = dict((
    ("dashboard", "Dashboard"),
    ("experiment", "Experiment"),
    ("investigation", "Investigation report"),
    ("survey", "Survey"),
    ("other", "Other"),
    ("", "")
))

AUTHORS = {
    "": " <>",
    "bmiroglio@mozilla.com": "Ben Miroglio <bmiroglio@mozilla.com>",
    "cdowhygelund@mozilla.com": "Corey Dow-Hygelund <cdowhygelund@mozilla.com>",
    "dxu@mozilla.com": "Danica Xu <dxu@mozilla.com>",
    "echo@mozilla.com": "Edward Cho <echo@mozilla.com>",
    "ethompson@mozilla.com": "Emily Thompson <ethompson@mozilla.com>",
    "flawrence@mozilla.com": "Felix Lawrence <flawrence@mozilla.com>",
    "gkaberere@mozilla.com": "George Kaberere <gkaberere@mozilla.com>",
    "jhollek@mozilla.com": "Julie Hollek <jhollek@mozilla.com>",
    "jmccrosky@mozilla.com": "Jesse McCrosky <jmccrosky@mozilla.com>",
    "ksiegler@mozilla.com": "Kimmy Siegler <ksiegler@mozilla.com>",
    "loines@mozilla.com": "Leif Oines <loines@mozilla.com>",
    "mgorlick@mozilla.com": "Marissa Gorlick <mgorlick@mozilla.com>",
    "mgrant@mozilla.com": "Marshall Grant <mgrant@mozilla.com>",
    "mmccorquodale@mozilla.com": "Megan McCorquodale <mmccorquodale@mozilla.com>",
    "mreid@mozilla.com": "Mark Reid <mreid@mozilla.com>",
    "rharter@mozilla.com": "Ryan Harter <rharter@mozilla.com>",
    "sghose@mozilla.com": "Saptarshi Ghose <sghose@mozilla.com>",
    "sguha@mozilla.com": "Saptarshi Guha <sguha@mozilla.com>",
    "shong@mozilla.com": "Su-Young Hong <shong@mozilla.com>",
    "tbrooks@mozilla.com": "Teon Brooks <tbrooks@mozilla.com>",
    "tismith@mozilla.com": "Tim D. Smith <tdsmith@mozilla.com>",
    "wbeard@mozilla.com": "Chris Beard <wbeard@mozilla.com>",
    "xluo@mozilla.com": "Xuan Luo <xluo@mozilla.com>",
}

column_map = {
    "jira_ticket": "Jira ticket",
    "Edit": "Edit",
    "completed_date": "Date",
    "title": "Title",
    "deliverable": "Link",
    "abstract": "Summary",
    "assignee": "Author",
    "reporter": "Reporter",
    "artifact": "Artifact",
    "product": "Product",
    "topic": "Topics",
    "tags": "Tags",
}

In [ ]:
spreadsheet = annotated.rename(columns=column_map)[column_map.values()]
spreadsheet["Topics"] = spreadsheet["Topics"].str.split(",").map(lambda x: (TOPICS[t] for t in x)).str.join(", ")
spreadsheet["Artifact"] = spreadsheet["Artifact"].map(lambda x: ARTIFACTS[x])
spreadsheet["Product"] = spreadsheet["Product"].map(lambda x: PRODUCT_CHOICES[x])
spreadsheet["Author"] = spreadsheet["Author"].map(lambda x: AUTHORS[x])

In [ ]:
spreadsheet.head()

In [ ]:
spreadsheet.to_csv("spreadsheet.csv", index=False)